In [1]:
import json
import os
import xml.etree.ElementTree as ET
import sys
import pandas as pd

In [2]:
'''
Create paths and folders
'''

# Folders with xml files
path_to_pubmed_files = os.path.abspath('../data/pubmed/')

# Path for json folder [json_test for debugging]
path_to_json_file = os.path.abspath('../data/json/')

# Variable for all parsed files
all_parsed_files = []

#substitute first file with main xml folder
all_folders = [path_to_pubmed_files, path_to_json_file]

print('\n-------------------------\n')
print('\nPaths and folders created\n')


-------------------------


Paths and folders created



In [3]:
#Function for creating folders

create_folders_errors = []

def create_folders(paths=[]):
    for p in paths:
        try:
            os.mkdir(p)
            if p:
                print('{} created'.format(p))
        except IOError as e:
            create_folders_errors.append(e)
            pass

        print('\n-------------------------\n')
        print('\nFolders created. Except: {}\n'.format(create_folders_errors))

In [4]:
create_folders(all_folders)


-------------------------


Folders created. Except: [FileExistsError(17, 'File exists')]


-------------------------


Folders created. Except: [FileExistsError(17, 'File exists'), FileExistsError(17, 'File exists')]



In [5]:
'''
Parse all xml files and save them in all_parsed_files.
Slow. Checks for xml files
'''

def parse_xml_files(path_to_folder):
    for filename in os.listdir(path_to_folder):
        # if not filename.endswith('.xml'): continue
        fullname = os.path.join(path_to_folder, filename)
        all_parsed_files.append(ET.parse(fullname).getroot())

        # Checking message to the terminal
        print("{} file parsed\n".format(fullname))

    print('Number of files parsed: {}\n'.format(len(all_parsed_files)))

    return all_parsed_files

In [6]:
parse_xml_files(path_to_pubmed_files)

ParseError: not well-formed (invalid token): line 1, column 0 (<string>)

In [ ]:
'''
Find values by unique tags in XML files
Save them in a dictionary
Export them to a json file
'''

all_data_dictionary = {}
print('\n-------------------------\n')
print('\nCreated all_data_dictionary\n')

In [ ]:
#function to create dictionary from tags
def create_dictionary_from_tag(tag, parsed_files = all_parsed_files, name_dictionary = all_data_dictionary):

    # Variable to store values
    # keys = []
    values = []

    # Iterate through all xml parsed files and tags, and append data to values
    for n in parsed_files:
        for i in n.iter(tag):
            if i.text != 0:
                values.append(i.text)
            else:
                values.append('nan')
            print("{} file parsed\n".format(i.text))

    # Create dictionary and set tags as keys
    name_dictionary.setdefault(tag, [])

    # Append values to dictionary
    for i in values:
        name_dictionary[tag].append(i)
        print("{} file added to the dictionary\n".format(i))

In [ ]:
create_dictionary_from_tag('Title')
create_dictionary_from_tag('ISOAbbreviation')
create_dictionary_from_tag('ArticleTitle')
create_dictionary_from_tag('AbstractText')
create_dictionary_from_tag('AuthorList/Author/Affiliation')
create_dictionary_from_tag('Language')
create_dictionary_from_tag('PublicationStatus')
create_dictionary_from_tag('ArticleIDLists/ArticleId')
create_dictionary_from_tag('Reference/Citation')

In [ ]:
# Function for adding nested tags

def add_new_tags(new_key):
    all_data_dictionary.setdefault(new_key, [])

    for n in all_parsed_files:
            value_conditions = n.find(new_key)
            if n.find(new_key) is not None:
                all_data_dictionary[new_key].append(value_conditions.text)
            else:
                all_data_dictionary[new_key].append('None')

    print('{} tag added'.format(new_key))

In [ ]:
add_new_tags('PubDate/Year')

In [ ]:
def check_values_key():
    print('\nChecking values by key\n')

    # Check number of values by key
    for key, value in all_data_dictionary.items():
        print(key, len(list(filter(bool, value))))

In [ ]:
check_values_key()

## Put all data from dictionary in a df

In [ ]:
df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in all_data_dictionary.items()]))

In [ ]:
pd.set_option('display.max_columns', 500)
df

In [ ]:
df.to_json('../data/json/sample_json.json')